In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from tkinter import *
from tkinter import filedialog, messagebox, simpledialog

class SkincareAnalysisApp:
    def __init__(self, master):
        self.master = master
        master.title("Skincare Product Analysis")

        # UI Title
        self.label = Label(master, text="Welcome to Skincare Product Analysis", font=("Arial", 14))
        self.label.pack(pady=10)

        # Load CSV File Button
        self.load_button = Button(master, text="Load CSV File", command=self.load_data, bg="lightblue")
        self.load_button.pack(pady=5)

        # Overall Analysis Button (Initially Disabled)
        self.analyze_button = Button(master, text="Overall Data Analysis", command=self.overall_analysis, state=DISABLED, bg="lightgreen")
        self.analyze_button.pack(pady=5)

        # Dropdown for Skin Type Selection
        self.skin_type_var = StringVar(master)
        self.skin_type_var.set("Select Skin Type")
        self.skin_type_menu = OptionMenu(master, self.skin_type_var, "normal", "oily", "dry", "sensitive", "combination")
        self.skin_type_menu.pack(pady=5)

        # Skin Type Analysis Button (Initially Disabled)
        self.skin_type_button = Button(master, text="Skin Type Analysis", command=self.skin_type_analysis, state=DISABLED, bg="lightyellow")
        self.skin_type_button.pack(pady=5)

        # Effectiveness Analysis Button (Initially Disabled)
        self.effectiveness_button = Button(master, text="Effectiveness Analysis", command=self.effectiveness_analysis, state=DISABLED, bg="lightcoral")
        self.effectiveness_button.pack(pady=5)

        # Comparison Table Button (Initially Disabled)
        self.comparison_button = Button(master, text="Comparison Table", command=self.comparison_table, state=DISABLED, bg="lightpink")
        self.comparison_button.pack(pady=5)

        # Sorting Button (Initially Disabled)
        self.sorting_button = Button(master, text="Sorting", command=self.sorting, state=DISABLED, bg="lightgray")
        self.sorting_button.pack(pady=5)

        # Text Display for Output
        self.text_display = Text(master, height=15, width=100, wrap=NONE)
        self.text_display.pack(pady=10)
        self.scroll_x = Scrollbar(master, orient=HORIZONTAL, command=self.text_display.xview)
        self.scroll_x.pack(fill=X)
        self.text_display.configure(xscrollcommand=self.scroll_x.set)

        self.data = None

    def load_data(self):   #LOAD THE DATA
        file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
        if file_path:
            self.data = pd.read_csv(file_path)
            messagebox.showinfo("Success", "Data loaded successfully.")

            # Enable analysis buttons after data is loaded
            self.analyze_button.config(state=NORMAL)
            self.skin_type_button.config(state=NORMAL)
            self.effectiveness_button.config(state=NORMAL)
            self.comparison_button.config(state=NORMAL)
            self.sorting_button.config(state=NORMAL)
#OVERALL ANALYSIS
    def overall_analysis(self):
        if self.data is not None:
            total_products = len(self.data)
            avg_rating = self.data["Ratings"].mean()
            most_common_concern = self.data["Concern"].mode()[0]
            avg_price = self.data["Price"].mean()
            result = f"Total Products: {total_products}\nAvg Rating: {avg_rating:.2f}\nMost Common Concern: {most_common_concern}\nAvg Price: ${avg_price:.2f}"
            messagebox.showinfo("Overall Analysis", result)
        else:
            messagebox.showwarning("Warning", "Please load data first.")

#SKIN TYPE ANALYSIS
    def skin_type_analysis(self):
        if self.data is not None:
            selected_skin_type = self.skin_type_var.get()
            if selected_skin_type == "Select Skin Type":
                messagebox.showwarning("Warning", "Please select a valid skin type.")
                return

            filtered_data = self.data[self.data["Skin type"].str.contains(selected_skin_type, case=False, na=False)]
            if not filtered_data.empty:
                avg_rating = filtered_data["Ratings"].mean()
                avg_price = filtered_data["Price"].mean()
                result = f"Avg Rating for {selected_skin_type} skin: {avg_rating:.2f}\nAvg Price: ${avg_price:.2f}"
                messagebox.showinfo(f"{selected_skin_type} Skin Analysis", result)
            else:
                messagebox.showwarning("Warning", f"No data found for {selected_skin_type} skin type.")
        else:
            messagebox.showwarning("Warning", "Please load data first.")

#EFFECTIVENESS ANALYSIS
    def effectiveness_analysis(self):
        if self.data is not None:
            # Histogram of Ratings
            plt.hist(self.data["Ratings"], bins=5, color="red", edgecolor="black", alpha=0.7, linewidth=3, label="Product Ratings")
            plt.xlabel("Ratings")
            plt.ylabel("Frequency")
            plt.title("Distribution of Skincare Product Ratings")
            plt.show()

            # Pie Chart of Most Common Skin Type
            categories = ["normal", "oily", "dry", "sensitive", "combination"]
            skintype_counts = {cat: self.data['Skin type'].str.contains(cat, case=False, na=False).sum() for cat in categories}
            skintype_count_df = pd.DataFrame(list(skintype_counts.items()), columns=['Skin Type', 'Count'])
            plt.pie(skintype_count_df["Count"], labels=skintype_count_df['Skin Type'], autopct="%1.1f%%", startangle=90)
            plt.title("Distribution of Skin Types")
            plt.show()

            # Bar Chart for Most Common Skin Concerns
            df1 = self.data.copy()
            df1['Concern'] = df1['Concern'].str.split(',')
            df1 = df1.explode('Concern')
            df1['Concern'] = df1['Concern'].str.strip()
            concern_counts = df1['Concern'].value_counts()
            plt.figure(figsize=(17, 8))
            plt.bar(concern_counts.index, concern_counts.values, color='skyblue')
            plt.xlabel('Skin Concern')
            plt.ylabel('Frequency')
            plt.title('Most Common Skin Concerns')
            plt.show()

            # Histogram of Price Distribution
            plt.figure(figsize=(8, 5))
            plt.hist(self.data['Price'], bins=20, color='purple', edgecolor='black')
            plt.xlabel('Price')
            plt.ylabel('Frequency')
            plt.title('Price Distribution of Skincare Products')
            plt.show()
        else:
            messagebox.showwarning("Warning", "Please load data first.")

#COMPARISION TABLE
    def comparison_table(self):
        if self.data is not None:
            selected_products = simpledialog.askstring("Input", "Enter product names separated by commas:")
            if not selected_products:
                return
            selected_products = [p.strip() for p in selected_products.split(",")]
            comparison_df = self.data[self.data["Product"].isin(selected_products)]
            if not comparison_df.empty:
                self.display_table(comparison_df)
            else:
                messagebox.showwarning("Warning", "No matching products found.")
        else:
            messagebox.showwarning("Warning", "Please load data first.")

#SORT THE VALUES
    def sorting(self):
        if self.data is not None:
            sorting_option = simpledialog.askstring("Sorting", "Enter sorting option: price_low, price_high, rating_low, rating_high")
            if sorting_option == "price_low":
                sorted_df = self.data.sort_values(by="Price", ascending=True)
            elif sorting_option == "price_high":
                sorted_df = self.data.sort_values(by="Price", ascending=False)
            elif sorting_option == "rating_low":
                sorted_df = self.data.sort_values(by="Ratings", ascending=True)
            elif sorting_option == "rating_high":
                sorted_df = self.data.sort_values(by="Ratings", ascending=False)
                
            else:
                messagebox.showwarning("Warning", "Invalid sorting option.")
                return
            self.display_table(sorted_df)
        else:
            messagebox.showwarning("Warning", "Please load data first.")

    def display_table(self, df):
        self.text_display.delete(1.0, END)
        self.text_display.insert(END, df.to_string(index=False))

if __name__ == "__main__":
    root = Tk()
    app = SkincareAnalysisApp(root)
    root.mainloop()
